# Vecinos más cercanos (K-NN) y Balance Sesgo - Varianza

Proponemos explorar el balance entre sesgo y varianza construyendo modelos de diferente complejidad empleando la aproximación por vecinos más cercanos. Asimismo, exploraremos este balance en ejercicios complementarios basados en árboles de decisión para distintas profundidades. Primero, analizaremos el balance en modelos de regresión y luego con clasificadores.   

## Regresión por vecinos más cercanos

### Atributos numéricos

Vamos a definir y ajustar un modelo de regresión por vecinos más cercanos para la variable `body_mass_g` del conjunto de datos de pingüinos. En primer lugar, solamente consideraremos los atributos numéricos.

In [ ]:
# Permite identificar donde estamos parados para 
# luego escribir el PATH correcto para acceder al conjunto de datos
import os
print(os.getcwd())

Al igual que en las guías anteriores, vamos a leer el conjunto de datos de pingüinos `penguins_size.csv`. Recordamos que contiene algunos `NA` que debemos eliminar... Y, además, hay una muestra con el atributo `sex` definido como "." que también borraremos...

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pgs = pd.read_csv('path_to/penguins_size.csv')
# COMPLETAR
# Remover NA y .



In [ ]:
# Confirmamos la distribución de muestras por especie
# Presenta un balance aceptable?
pgs['species'].value_counts()

Ahora, nos quedamos solamente con los atributos numéricos y separamos en datos de entrenamiento y prueba. Recordamos que nuestro *target* será `body_mass_g`

In [4]:
from sklearn.model_selection import train_test_split
# COMPLETAR
# X = 
# y = 
# X_train, X_test, y_train, y_test = 



Vamos a generar una instancia de la clase `KNeighborsRegressor()` que nos permitirá realizar un ajuste por vecinos más cercanos. Asimismo, es fundamental reescalar los atributos para que las *distancias* en las diferentes direcciones sean comparables incorporando previamente una transformación de la clase `StandardScaler()`. Dado que siempre ambos procesos irán encadenados, es conveniente introducir un `Pipeline`.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline

# COMPLETAR
# Definir un Pipeline que encade un StandardScaler y un KNeighborsRegressor
# 
# pipe = 
 
pipe.get_params()

Ahora, podemos operar con el *pipe* como si se tratara de un estimador regular. Es decir, podemos aplicar al *pipe* los métodos `.fit()`, `.predict()`, o cualquier otro, en el orden correspondiente. Particularmente, nos interesa estudiar el balance sesgo-varianza. En este caso, podemos definir un ciclo que itere sobre el número de primeros vecinos y almacenar los resultados de algunas métricas tanto en entrenamiento como en prueba. Por ejemplo, podemos tomar el método `.score()` que recupera el valor del coeficiente de determinación ($R^2$).

Para cada iteración, debemos modifcar el número de vecinos más cercanos. Como vimos, podemos acceder a esta configuración a través del método `.set_params()` y pasando como argumento `knn__n_neighbors=k`. En este caso, `k` representa el número de vecinos más cercanos y la variable o *key* se construye como "nombre_del_paso + doble guión bajo (__) + nombre del parámetro".  En este ejemplo elegimos como nombre del segundo paso del `Pipeline` `knn`.

Sugerimos para `k` valores en el rango entre 1 y 50.

In [6]:
from sklearn.metrics import root_mean_squared_error

k_all = np.arange(2,41, 2)
r2_train = np.array([])
r2_test = np.array([])
# rmse_train 
# rmse_test
for k in k_all:
    pipe.set_params(knn__n_neighbors=k)
    pipe.fit(X_train, y_train)
    r2_train = np.append(r2_train, [pipe.score(X_train, y_train)])
    r2_test = np.append(r2_test, [pipe.score(X_test, y_test)])
    # rmse_train = 
    # rmse_test = 


En este ejemplo, contamos con los $R^2$ de entrenamiento y prueba en función del número de vecinos. Así, podemos trazar ambas curvas que usualmente se representan en función de 1/k. 

**Ejercicio**: almacenar simultaneamente los RMSE de entrenamiento y prueba en función del número de vecinos. Realizar el gráfcio correspondiente.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows = 1,ncols = 2, dpi=100)
ax[0].plot(1/k_all, rmse_train, label='Train', linestyle='-.')
ax[0].plot(1/k_all, rmse_test, label='Test', linestyle='-')
ax[0].set_xlim([0,np.max(1/k_all)])
ax[0].set_xlabel("1/k")
ax[0].set_ylabel("RMSE (gr)")
ax[0].legend(loc='lower left')

ax[1].plot(1/k_all, r2_train, label='Train', linestyle='-.')
ax[1].plot(1/k_all, r2_test, label='Test', linestyle='-')
ax[1].set_xlim([0,np.max(1/k_all)])
ax[1].set_xlabel("1/k")
ax[1].set_ylabel("R2")
ax[1].legend(loc='lower left')

**Ejericio**: Señalar en las figuras anteriores las zonas de sobreajuste (over-fitting) y subajuste (under-fitting). Explicar cómo se las encuentra. ¿Cuál podría ser un rango de `k` óptimo? 

### Atributos numéricos y categóricos

Proponemos repetir el estudio del balance sesgo-varianza pero ahora también incluyendo las variables categóricas `species` y `sex` en el análisis. Separamos en datos de entrenamiento y prueba recordando que nuestro *target* será `body_mass_g`. Asimismo, construimos listas con los nombres de los atriutos numéricos y categóricos que serán usadas más adelante

In [ ]:
pgs_numeric = pgs.select_dtypes(include='number').columns.to_list()
pgs_numeric.remove('body_mass_g')
pgs_cat = ['species', 'sex']
pgs_drop = ['island']
print(pgs_numeric)

# COMPLETAR
# X = 
# y = 
# X_train, X_test, y_train, y_test = 

Notamos que las variables numéricas y categóricas no pueden tratarse de la misma manera. Por un lado, continuaremos reescalando los datos cuantitativos usando la clase `StandardScaler()`. Por otro lado, codificaremos los datos cualitativos aplicando una transformación por `OneHotEncoder()`. Ambas operaciones se pueden incluir en un mismo preprcesamiento con la ayuda de `ColumnTransformer`. Finalmente, este preprocesamiento se encadena con la regresión por vecinos más cercanos mediante `Pipeline`. El objetivo es que los datos sean transformados, según su tipo, antes de llegar al regresor. 

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preproc = ColumnTransformer(
    [('scaler', StandardScaler(), pgs_numeric),
     ('one-hot',OneHotEncoder(), pgs_cat),
     ('drop', 'drop', pgs_drop),
    ])

pipe = Pipeline([('preproc', preproc), ('knn', KNeighborsRegressor())]) 
pipe

De manera análoga, buscamos realizar gráficos para las distintas métricas consideradas, tanto con datos de entrenamiento como de prueba. Para ello, nos planteamos un ciclo que itera sobre un número creciente de primeros vecinos (en el rango entre 1 y 50). 

In [10]:
from sklearn.metrics import root_mean_squared_error

# COMPLETAR
# Realizar un ciclo sobre el número de vecinos K, almacenar las respectivas métricas
# k_all = 
# rmse_train = 
# rmse_test = 
# for k in k_all:
    # Configurar el número de vecinos en k

    # Ajustar y calcular las métricas para entrenamiento y prueba

    # Alamcenar K, rmse_train y rmse_test

In [ ]:
# COMPLETAR
# Elaborar una figura que trace los RMSE de prueba y entrenamiento en función de 1/k




**Ejercicio**: Identificar regiones de sobre y subajuste. ¿Se podría afirmar que el modelo que toma variables numéricas y categóricas tiene un mejor desempeño que el que solo emplea datos numéricos? Repetir el análisis usando `DecisionTreeRegressor` en función de diferentes profundidades.

## Clasificación por vecinos más cercanos

Vamos a definir y ajustar un modelo de clasificación por vecinos más cercanos para la variable `species` del conjunto de datos de pingüinos. En este caso consideraremos solamente el atributos numérico `flipper_length_mm` y, adicionalmente, la variable categórica `sex`. Deberemos:

1. Separa en datos de entrenamiento y prueba.
2. Elaborar una transformación por columnas que aplique la clase `StandardScaler` a los datos numéricos y la clase `OneHotEncoder` a los cateóricos.
3. Elaborar un pipeline que encadene el preprocesado por columnas y, luego, inicie el proceso de `KNeighborsClassifier()`. 


In [ ]:
pgs_numeric = ['flipper_length_mm']
pgs_cat = ['sex']

print(pgs_numeric)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# COMPLETAR
# X = 
# y = 
# X_train, X_test, y_train, y_test = 

# Definir un preprocesado pór columnas que aplique 
# StandardScaler() en la variable numérica
# OneHotEncoder() en la variable categórica

# preproc = ColumnTransformer(
#
#
#    )

# Definir un Pipeline que encadene el
# preprocesado y el clasificador de primeros vecinos

# pipe = 
pipe.get_params()

In [ ]:
X_train.head()

El análisis del balance sesgo-varianza para los clasificadores conduce, al igual que antes, a la puesta en práctica de un ciclo que recorre valores crecientes de `k` para vecinos más cercanos y, calcula y almacena la exactitud (`accuracy_score`). Dado que las tres especies de pingüinos se presentan en cantidades iguales, la exactitud es una métrica adecuada para evaluar el desempeño general de los clasificadores. Debemos recordar que otras métricas, como el recall o precision deberían calcularse por especie ya que operamos con tres clases diferentes.  

In [15]:

from sklearn.metrics import accuracy_score

# COMPLETAR
# Realizar un ciclo sobre el número de vecinos K, almacenar la respectiva métrica
# k_all = 
# acc_train =
# acc_test = 
for k in range(1,30):
    # Configurar el número de vecinos en k

    # Ajustar y calcular la métrica para entrenamiento y prueba

    # Alamcenar K, acc_train y acc_test
    

Los valores registrados de exactitud en función del número de primeros vecinos, tanto para los conjuntos de pureba como entrenamiento nos habilitan el trazado de una figura que representa el balance sesgo-varianza.

In [ ]:
# COMPLETAR
# Elaborar una figura que trace los Accuracy de prueba y entrenamiento en función de 1/k




**Pregunta**: Sabiendo que el atributo `sex` es binario, ¿podríamos pasar como argumento de `OneHotEncoder` la opción `drop='first'`? ¿Tiene impacto en el resultado?

**Ejercicio**: Reconocer zonas de sobre y subajuste. Elegir un `k`, número de primeros vecinos, que considere ofrezca el mejor balance sesgo-varianza. Volver a ajustar el modelo para dicho `k` e imprimir las predicciones para el conjunto de prueba. Notar que la predicción es el nombre de la especie correspondiente a cada muestra o dato.

In [ ]:
# COMPLETAR
# elegir el K que considere alcanza el mejor balance entre sesgo y varianza
# k =

# pipe.set_params(knn__n_neighbors=k)
# Ajustar el modelo en el nuevo k con datos de entrenamiento

# Realizar la predicción correspondiente para datos de prueba
# y_pred_test = 
print(y_pred_test)


**Ejercicio**: Calcular la matriz de confusión para `k` cuando toma el valor que se considera realiza el mejor balance entre sesgo y varianza. Identificar que especies confunde el clasificador en mayor cantidad.

In [ ]:
from sklearn.metrics import confusion_matrix
# COMPLETAR
# Imprimir la matriz de confusión para los datos de prueba con el k óptimo


**Ejercicio**: Representar en una figura las especies que serían predecidas cuando `flipper_length_mm`varía en el rango entre 160 y 240, para ambos sexos. Superponer con lospuntos de entrenamiento. Observar como cambia la figura con la elección del número de vecinos más cercanos.

In [ ]:
# COMPLETAR 
# Realizar la representación de especies en función de
# los atributos flipper_length_mm y sex

# Armar un DataFrame que recorra los valores 
# de flipper_length_mm desde el mínimo hasta el máximo
# y el atributo sex en sus valores Male y FEMALE

# Predecir las especies 

# Considerar los métodos de seaborn 
# catplot, stripplot o scatterplot para la representación 
# de las especies en función del atributo 



**Ejercicio**: Explorar cómo afecta la clasificación si los *targets* se pesan por la inversa de la distancia al punto.

**Ejercicio**: Repetir el análisis usando `DecisionTreeClassifier` en función de diferentes profundidades.